# Serving a TensorFlow Model as a REST Endpoint with TensorFlow Serving and SageMaker

We need to understand the application and business context to choose between real-time and batch predictions. Are we trying to optimize for latency or throughput? Does the application require our models to scale automatically throughout the day to handle cyclic traffic requirements? Do we plan to compare models in production through A/B tests?

If our application requires low latency, then we should deploy the model as a real-time API to provide super-fast predictions on single prediction requests over HTTPS. We can deploy, scale, and compare our model prediction servers with SageMaker Endpoints.

## Interesting Reads
* [**How Roblox Scaled BERT to Serve 1+ Billion Daily Requests on CPUs**](https://blog.roblox.com/2020/05/scaled-bert-serve-1-billion-daily-requests-cpus/)

<img src="img/sagemaker-architecture.png" width="80%" align="left">

In [1]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

In [2]:
%store -r training_job_name

In [3]:
try:
    training_job_name
    print('[OK]')
except NameError:
    print('+++++++++++++++++++++++++++++++')
    print('[ERROR] Please run the notebooks in the previous TRAIN section before you continue.')
    print('+++++++++++++++++++++++++++++++')

[OK]


In [4]:
print(training_job_name)

tensorflow-training-2020-09-26-18-44-25-975


# Copy the Model to the Notebook

In [5]:
!aws s3 cp s3://$bucket/$training_job_name/output/model.tar.gz ./model.tar.gz

download: s3://sagemaker-us-west-2-085964654406/tensorflow-training-2020-09-26-18-44-25-975/output/model.tar.gz to ./model.tar.gz


In [6]:
!mkdir -p ./model/
!tar -xvzf ./model.tar.gz -C ./model/

transformers/
transformers/fine-tuned/
transformers/fine-tuned/tf_model.h5
transformers/fine-tuned/config.json
tensorflow/
tensorflow/saved_model/
tensorflow/saved_model/0/
tensorflow/saved_model/0/variables/
tensorflow/saved_model/0/variables/variables.index
tensorflow/saved_model/0/variables/variables.data-00000-of-00001
tensorflow/saved_model/0/assets/
tensorflow/saved_model/0/saved_model.pb
code/
code/inference.py
tensorboard/
metrics/
metrics/confusion_matrix.png


In [7]:
!saved_model_cli show --all --dir ./model/tensorflow/saved_model/0/

2020-09-26 20:12:49.848844: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.0/lib64:/usr/local/cuda-10.0/extras/CUPTI/lib64:/usr/local/cuda-10.0/lib:/usr/local/cuda-10.0/efa/lib:/opt/amazon/efa/lib:/opt/amazon/efa/lib64:/usr/lib64/openmpi/lib/:/usr/local/lib:/usr/lib:/usr/local/mpi/lib:/lib/:/usr/lib64/openmpi/lib/:/usr/local/lib:/usr/lib:/usr/local/mpi/lib:/lib/:/usr/lib64/openmpi/lib/:/usr/local/lib:/usr/lib:/usr/local/mpi/lib:/lib/:
2020-09-26 20:12:49.848926: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.0/lib64:/usr/local/cuda-10.0/extras/CUPTI/lib64:/usr/local/cuda-10.0/lib:/usr/local/cuda-10.0/ef

# Show `inference.py`

In [8]:
!pygmentize ./model/code/inference.py

import json
import subprocess
import sys
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'tensorflow==2.1.0'])
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'transformers==2.8.0'])
import tensorflow as tf
from transformers import DistilBertTokenizer

classes=[1, 2, 3, 4, 5]

max_seq_length=64

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

def input_handler(data, context):
    transformed_instances = []

    print('DATA {}'.format(data))

    for instance in data:
        data_str = instance.decode('utf-8')
        print('DATA_STR {}'.format(data_str))
        
        tokens = tokenizer.tokenize(data_str)
        print('TOKENS {}'.format(tokens))

        encode_plus_tokens = tokenizer.encode_plus(data_str,
                                                   pad_to_max_length=True,
                                                   max_length=max_seq_length,
#                                                   truncation=True


# Deploy the Model
This will create a default `EndpointConfig` with a single model.  

The next notebook will demonstrate how to perform more advanced `EndpointConfig` strategies to support canary rollouts and A/B testing.

_Note:  If not using a US-based region, you may need to adapt the container image to your current region using the following table:_

https://docs.aws.amazon.com/deep-learning-containers/latest/devguide/deep-learning-containers-images.html

In [9]:
import time

timestamp = int(time.time())

tensorflow_model_name = '{}-{}-{}'.format(training_job_name, 'tf', timestamp)

print(tensorflow_model_name)

tensorflow-training-2020-09-26-18-44-25-975-tf-1601151176


In [10]:
from sagemaker.tensorflow.model import TensorFlowModel

tensorflow_model = TensorFlowModel(name=tensorflow_model_name,
                                   model_data='s3://{}/{}/output/model.tar.gz'.format(bucket, training_job_name),
                                   role=role,                
                                   framework_version='2.1.0')

In [11]:
tensorflow_endpoint_name = '{}-{}-{}'.format(training_job_name, 'tf', timestamp)

print(tensorflow_endpoint_name)

tensorflow-training-2020-09-26-18-44-25-975-tf-1601151176


In [12]:
tensorflow_model = tensorflow_model.deploy(endpoint_name=tensorflow_endpoint_name,
                                           initial_instance_count=1, # Should use >=2 for high(er) availability 
                                           instance_type='ml.m5.4xlarge', # requires enough disk space for tensorflow, transformers, and bert downloads
                                           wait=False)

In [13]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/endpoints/{}">SageMaker REST Endpoint</a></b>'.format(region, tensorflow_endpoint_name)))


# _Wait Until the Endpoint is Deployed_

In [14]:
%%time

waiter = sm.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=tensorflow_endpoint_name)

CPU times: user 140 ms, sys: 11.7 ms, total: 151 ms
Wall time: 6min 31s


# _Wait Until the ^^ Endpoint ^^ is Deployed_

# Test the Deployed Model

In [15]:
import json
from sagemaker.tensorflow.model import TensorFlowPredictor

predictor = TensorFlowPredictor(endpoint_name=tensorflow_endpoint_name,
                                sagemaker_session=sess,
                                model_name='saved_model',
                                model_version=0)

### Waiting for the Endpoint to be ready to Serve Predictions

In [16]:
import time

time.sleep(30)

# Predict the `star_rating` with Ad Hoc `review_body` Samples

In [17]:
reviews = ["This is great!"]

predicted_classes = predictor.predict(reviews)

for predicted_class, review in zip(predicted_classes, reviews):
    print('[Predicted Star Rating: {}]'.format(predicted_class), review)

[Predicted Star Rating: 5] This is great!


# Predict the `star_rating` with `review_body` Samples from our TSV's

In [18]:
import csv

df_reviews = pd.read_csv('./data/amazon_reviews_us_Digital_Software_v1_00.tsv.gz', 
                         delimiter='\t', 
                         quoting=csv.QUOTE_NONE,
                         compression='gzip')
df_sample_reviews = df_reviews[['review_body', 'star_rating']].sample(n=100)
df_sample_reviews = df_sample_reviews.reset_index()
df_sample_reviews.shape

(100, 3)

In [19]:
import pandas as pd

def predict(review_body):
    return predictor.predict(review_body)

df_sample_reviews['predicted_class'] = df_sample_reviews['review_body'].map(predict)
df_sample_reviews.head(5)

,index,review_body,star_rating,predicted_class
0,47054,Exactly what was advertized.,5,[3]
1,76847,After reading all the bad reviews I still boug...,1,[1]
2,51770,I feel its the best free one out there!,5,[5]
3,20692,Does the job.,5,[5]
4,99422,"IF YOU ARE SMART, YOU WILL JUST BUY THE DISKS!...",1,[1]


# Save for Next Notebook(s)

In [20]:
%store tensorflow_model_name

Stored 'tensorflow_model_name' (str)


In [21]:
%store tensorflow_endpoint_name 

Stored 'tensorflow_endpoint_name' (str)


In [22]:
%store

Stored variables and their in-db values:
auto_ml_job_name                                      -> 'automl-dm-26-16-00-25'
autopilot_endpoint_name                               -> 'automl-dm-ep-26-16-21-49'
autopilot_train_s3_uri                                -> 's3://sagemaker-us-west-2-085964654406/data/amazon
balance_dataset                                       -> True
experiment_name                                       -> 'Amazon-Customer-Reviews-BERT-Experiment-160114585
ingest_create_athena_db_passed                        -> True
ingest_create_athena_table_parquet_passed             -> True
ingest_create_athena_table_tsv_passed                 -> True
max_seq_length                                        -> 64
prepare_trial_component_name                          -> 'TrialComponent-2020-09-26-184417-oahc'
processed_test_data_s3_uri                            -> 's3://sagemaker-us-west-2-085964654406/sagemaker-s
processed_train_data_s3_uri                           -> 's3://sa

# Delete Endpoint
To save cost, we should delete the endpoint.

In [23]:
# sm.delete_endpoint(
#      EndpointName=tensorflow_endpoint_name
# )

In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();

<IPython.core.display.Javascript object>